In [3]:
import pandas as pd
import chardet
import re

In [4]:
extraction_dataframe = pd.read_pickle("master_import_dataframe.pkl")

First, design the schema.

Because the data is unstructured, we will identify key values via visual exploration and extract them via keyword recognition (regex)

In [5]:
extraction_dataframe.head()

,Complete Records
0,Complet...
1,Complet...
2,Complete...
3,Complet...
4,Complete...


Inspect data structure within instances here:

In [6]:
extraction_dataframe["Complete Records"][24]

individual_exploratory_instance = extraction_dataframe["Complete Records"][24]

Let's break out primary keys in the schema first. Here is the VIN:

There are two options here: Use RegEx with the dataframes imported as strings, or work with the dataframes themselves.

The second option means less transformation of data and thus less potential to lose data granularity, but it also means that objects within cells must be split eventually.

In [7]:
extraction_dataframe['Complete Records String'] =  extraction_dataframe['Complete Records'].apply(lambda x: x.to_string(index=False, header=False))

In [ ]:
extraction_dataframe.head()

In [8]:
# Define regex pattern for VIN
vin_pattern = re.compile(r"User Entered VIN,([0-9A-Za-z]+)")

# Apply regex pattern on the dataframe and create a new column 'VIN'
extraction_dataframe['VIN'] = extraction_dataframe['Complete Records String'].apply(lambda x: vin_pattern.search(x).group(1) if vin_pattern.search(x) else None)

extraction_dataframe.head()


,Complete Records,Complete Records String,VIN
0,Complet...,...,1FTBR1Y8XMKA57198
1,Complet...,...,1FDXE45S29DA10452
2,Complete...,...,1FDXE4FS3BDB02206
3,Complet...,...,1FDXE4FS4BDA50634
4,Complete...,...,1FDXE4FS5BDA96599


In [9]:
vin_only_dataframe = extraction_dataframe.copy()
vin_only_dataframe.drop(['Complete Records', 'Complete Records String'], axis = 1, inplace = True)

In [22]:
vin_only_dataframe.head()
vin_only_dataframe.to_pickle('working_files/vin_only_dataframe.pkl')

User information:

In [11]:
# Define regex pattern for VIN
#This regex pattern will stop at \r\n (a 'carriage return') because \ is not an included character in the search pattern.
user_pattern = re.compile(r"User,([A-Za-z' '.]+)")

# Apply regex pattern on the dataframe and create a new column 'VIN'
extraction_dataframe['User'] = extraction_dataframe['Complete Records String'].apply(lambda x: user_pattern.search(x).group(1) if user_pattern.search(x) else None)

extraction_dataframe.head()

,Complete Records,Complete Records String,VIN,User
0,Complet...,...,1FTBR1Y8XMKA57198,W Craig
1,Complet...,...,1FDXE45S29DA10452,M. Mikhailov
2,Complete...,...,1FDXE4FS3BDB02206,SP
3,Complet...,...,1FDXE4FS4BDA50634,M. Mikhailov
4,Complete...,...,1FDXE4FS5BDA96599,DENIS SHARSHUNSKIY


Case number breakout and some data cleaning:

In [12]:
case_number_pattern = re.compile(r"Case Number,([0-9-]+)")

extraction_dataframe['Case Number'] = extraction_dataframe['Complete Records String'].apply(lambda x: case_number_pattern.search(x).group(1) if case_number_pattern.search(x) else None)

extraction_dataframe.head()

#This step requires additional cleaning, as all case numbers begin with 0 but are not always included. We must iterate through the instances of extraction_dataframe['Case Number'] and check if not NaN, and if .startswith() == 0, then prepend 0.

,Complete Records,Complete Records String,VIN,User,Case Number
0,Complet...,...,1FTBR1Y8XMKA57198,W Craig,02951882-2023
1,Complet...,...,1FDXE45S29DA10452,M. Mikhailov,01371524-2019
2,Complete...,...,1FDXE4FS3BDB02206,SP,1047504-2018
3,Complet...,...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022
4,Complete...,...,1FDXE4FS5BDA96599,DENIS SHARSHUNSKIY,00871457


In [13]:
def has_seven_digits_before_hyphen(case_number):
    number_segment = case_number.split('-')[0]
    return len(number_segment) == 7

extraction_dataframe['Case Number'] = extraction_dataframe['Case Number'].apply(
    lambda x: '0' + x if pd.notna(x) and not x.startswith('0') and has_seven_digits_before_hyphen(x) else x
)
extraction_dataframe['Case Number']

0     02951882-2023
1     01371524-2019
2     01047504-2018
3     02596301-2022
4          00871457
5       866715-2017
6       547078-2014
7       685630-2015
8     01901400-2020
9       734406-2016
10             None
11    01358628-2019
12    01424658-2019
13    02464402-2021
14    01825864-2020
15    01144564-2018
16             None
17    02091914-2021
18    01869039-2020
19    02846539-2022
20    02083717-2020
21    02146598-2021
22    01199426-2018
23    02086865-2020
24    01858026-2020
25             None
26             None
27      924258-2018
28             None
29             None
30    01622036-2019
Name: Case Number, dtype: object

Collect EDR Data Imaging Date

In [15]:
imaging_date_pattern = re.compile(r"\d{2}/\d{2}/\d{4}")

extraction_dataframe['EDR Data Imaging Date'] = extraction_dataframe['Complete Records String'].apply(lambda x: imaging_date_pattern.search(x).group() if imaging_date_pattern.search(x) else None)

extraction_dataframe.head()

,Complete Records,Complete Records String,VIN,User,Case Number,EDR Data Imaging Date
0,Complet...,...,1FTBR1Y8XMKA57198,W Craig,02951882-2023,02/10/2023
1,Complet...,...,1FDXE45S29DA10452,M. Mikhailov,01371524-2019,02/20/2019
2,Complete...,...,1FDXE4FS3BDB02206,SP,01047504-2018,08/01/2018
3,Complet...,...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022,04/29/2022
4,Complete...,...,1FDXE4FS5BDA96599,DENIS SHARSHUNSKIY,00871457,08/12/2017


Filenames:

In [16]:
filename_pattern = re.compile(r"Filename,(?:.+?X )*?([^X]+\.CDRX)")

extraction_dataframe['Filename'] = extraction_dataframe['Complete Records String'].apply(lambda x: filename_pattern.search(x).group(1) if filename_pattern.search(x) else None)

extraction_dataframe.head()

,Complete Records,Complete Records String,VIN,User,Case Number,EDR Data Imaging Date,Filename
0,Complet...,...,1FTBR1Y8XMKA57198,W Craig,02951882-2023,02/10/2023,None
1,Complet...,...,1FDXE45S29DA10452,M. Mikhailov,01371524-2019,02/20/2019,None
2,Complete...,...,1FDXE4FS3BDB02206,SP,01047504-2018,08/01/2018,None
3,Complet...,...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022,04/29/2022,None
4,Complete...,...,1FDXE4FS5BDA96599,DENIS SHARSHUNSKIY,00871457,08/12/2017,None


Saved On Date:

In [17]:
#This is a difficult segment because we do not want to include the variable name 'Saved On.' Thus, we must filter for days of the work.
saved_on_pattern = re.compile(r"((?:Mon|Tue|Wed|Thu|Fri|Sat|Sun)day, .+? at \d{1,2}:\d{2}:\d{2})")

extraction_dataframe['Saved On'] = extraction_dataframe['Complete Records String'].apply(lambda x: saved_on_pattern.search(x).group() if saved_on_pattern.search(x) else None)

extraction_dataframe.head()

,Complete Records,Complete Records String,VIN,User,Case Number,EDR Data Imaging Date,Filename,Saved On
0,Complet...,...,1FTBR1Y8XMKA57198,W Craig,02951882-2023,02/10/2023,None,"Friday, February 10 2023 at 14:13:03"
1,Complet...,...,1FDXE45S29DA10452,M. Mikhailov,01371524-2019,02/20/2019,None,None
2,Complete...,...,1FDXE4FS3BDB02206,SP,01047504-2018,08/01/2018,None,None
3,Complet...,...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022,04/29/2022,None,"Friday, April 29 2022 at 13:07:54"
4,Complete...,...,1FDXE4FS5BDA96599,DENIS SHARSHUNSKIY,00871457,08/12/2017,None,None


CDR Version Info:

In [18]:
imaged_with_pattern = re.compile(r'(Crash Data Retrieval Tool \d+\.\d+)')

def extract_two_instances(text):
    matches = imaged_with_pattern.findall(text)
    first_instance = matches[0] if len(matches) >= 1 else None
    second_instance = matches[1] if len(matches) >= 2 else None
    return first_instance, second_instance

extraction_dataframe['Imaged with CDR Version:'], extraction_dataframe['Reported with CDR Version:'] = zip(*extraction_dataframe['Complete Records String'].apply(lambda x: extract_two_instances(x)))

extraction_dataframe.head()


,Complete Records,Complete Records String,VIN,User,Case Number,EDR Data Imaging Date,Filename,Saved On,Imaged with CDR Version:,Reported with CDR Version:
0,Complet...,...,1FTBR1Y8XMKA57198,W Craig,02951882-2023,02/10/2023,None,"Friday, February 10 2023 at 14:13:03",Crash Data Retrieval Tool 23.0,Crash Data Retrieval Tool 21.5
1,Complet...,...,1FDXE45S29DA10452,M. Mikhailov,01371524-2019,02/20/2019,None,None,Crash Data Retrieval Tool 17.10,Crash Data Retrieval Tool 21.5
2,Complete...,...,1FDXE4FS3BDB02206,SP,01047504-2018,08/01/2018,None,None,Crash Data Retrieval Tool 17.8,Crash Data Retrieval Tool 21.5
3,Complet...,...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022,04/29/2022,None,"Friday, April 29 2022 at 13:07:54",Crash Data Retrieval Tool 21.4,Crash Data Retrieval Tool 21.5
4,Complete...,...,1FDXE4FS5BDA96599,DENIS SHARSHUNSKIY,00871457,08/12/2017,None,None,Crash Data Retrieval Tool 17.3,Crash Data Retrieval Tool 21.5


In [19]:
extraction_dataframe.head()

,Complete Records,Complete Records String,VIN,User,Case Number,EDR Data Imaging Date,Filename,Saved On,Imaged with CDR Version:,Reported with CDR Version:
0,Complet...,...,1FTBR1Y8XMKA57198,W Craig,02951882-2023,02/10/2023,None,"Friday, February 10 2023 at 14:13:03",Crash Data Retrieval Tool 23.0,Crash Data Retrieval Tool 21.5
1,Complet...,...,1FDXE45S29DA10452,M. Mikhailov,01371524-2019,02/20/2019,None,None,Crash Data Retrieval Tool 17.10,Crash Data Retrieval Tool 21.5
2,Complete...,...,1FDXE4FS3BDB02206,SP,01047504-2018,08/01/2018,None,None,Crash Data Retrieval Tool 17.8,Crash Data Retrieval Tool 21.5
3,Complet...,...,1FDXE4FS4BDA50634,M. Mikhailov,02596301-2022,04/29/2022,None,"Friday, April 29 2022 at 13:07:54",Crash Data Retrieval Tool 21.4,Crash Data Retrieval Tool 21.5
4,Complete...,...,1FDXE4FS5BDA96599,DENIS SHARSHUNSKIY,00871457,08/12/2017,None,None,Crash Data Retrieval Tool 17.3,Crash Data Retrieval Tool 21.5


In [21]:
extraction_dataframe.to_pickle("working_files/extraction_dataframe_with_keys.pkl")